# Assignment 2

## Section 1

In [2]:
#Import aall libraries
import pyspark
import findspark
import pandas as pd
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.ml.feature import StopWordsRemover, HashingTF, IDF, StringIndexer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
#VectorAssembler provides transformer to convert Dataframe columns into vectors
from pyspark.ml.feature import VectorAssembler
#StringIndexer provides transformer to convert string labels into numerical values
from pyspark.ml.feature import StringIndexer

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession

#N.B. before proceeding you must change the path passed to the findspark.init function to the location where spark is installed on your machine
findspark.init('C:/Spark/spark-3.5.3-bin-hadoop3')

In [ ]:
#read the dataset into a dataframe named 'df'
df=pd.read_csv("C:/Users/usman/Desktop/Master/Big Data Analytics/Assignment 2/Dataset/kindle_reviews.csv")


In [ ]:
#shows the first 5 rows of the unamended dataset
df.head(5)

In [ ]:
#count number of rows for each column
df.count()

#get infromation about the columns
df.info()

#get further statistical information about the dataframe
df.describe()

In [ ]:
#check how many null values there are in columns
df.isnull().sum()

#Remove all the rows with missing values that adds no information into a new df
new_df=df.dropna(subset=['reviewText'])

#renove the unnamed column (amended new_df)
new_df = new_df[['asin','helpful','overall','reviewText','reviewTime','reviewerID','reviewerName','summary','unixReviewTime']]

#check to see if any unknown missing values in amended dataframe
new_df.isnull().sum()
#new_df.head(5)

In [ ]:
#code sourced from (Serdar OZTURK): https://www.kaggle.com/code/serdarsozturk/rating-sorting

new_features = new_df["helpful"].str.split(",", expand=True)
new_features.dtypes

new_df["helpful_yes"] = new_features[0].str.lstrip("[")
new_df["helpful_yes"] = new_df["helpful_yes"].astype("int64")
new_df["helpful_yes"].head()

new_df["total_vote"] = new_features[1].str.rstrip("]")
new_df["total_vote"] = new_df["total_vote"].astype("int64")
new_df['total_vote'].head()

new_df["helpful_no"] = new_df["total_vote"] - new_df["helpful_yes"]
new_df["helpful_no"].head()

#change datatype of reviewTime Colum
df.reviewTime=pd.to_datetime(df.reviewTime)
#add a new 'year column to the dataframe
new_df['year']=new_df.reviewTime.dt.year

#adds a new column based on overall rating
new_df['review_positive_negative'] = new_df['overall'].apply(lambda x: 1 if x >= 3 else 0)

new_df.head(20)

In [ ]:
#histogram for overall rating
hist = new_df['overall'].hist(bins=5) 
   
plt.title('Histogram for overall Column') 
plt.xlabel('overall rating') 
plt.ylabel('Frequency') 
plt.show()


#bar plot to check number of reviews per year
new_df['year'].value_counts().sort_index().plot(kind='bar')
plt.title('Reviews per Year')
plt.xlabel('Year')
plt.ylabel('amount of reviews')
plt.show()

#countplot for ratings
sns.countplot(x='overall', data=new_df)
plt.xlabel('Rating') 
plt.ylabel('Count') 
plt.title('Count Plot of Ratings')
plt.show()


In [ ]:
#seperate top 5 non helpful ratings into a new df
helpfulno_df = new_df['helpful_no'].value_counts().nlargest(5).index
helpfulno_df = new_df[new_df['helpful_no'].isin(helpfulno_df)]

#countplot for helpful no
sns.countplot(x='helpful_no', data=helpfulno_df) 
plt.xlabel('Not Helpful') 
plt.ylabel('Count') 
plt.title('Count plot for number of unhelpful Reviews')
plt.show()

#seperate top 5 helpful ratings into a new df
helpfulyes_df = new_df['helpful_yes'].value_counts().nlargest(5).index
helpfulyes_df = new_df[new_df['helpful_yes'].isin(helpfulyes_df)]

#countplot for helpful yes
sns.countplot(x='helpful_yes', data=helpfulyes_df)
plt.xlabel('Yes Helpful') 
plt.ylabel('Count') 
plt.title('Count plot for number of helpful reviews')
plt.show()

In [ ]:
#countplot for whether a review is positive or negative
sns.countplot(x='review_positive_negative', data=new_df)
plt.xlabel('Positive / Negative Reviews') 
plt.ylabel('Count') 
plt.title('Count plot for whether a review is positive or negative (1=yes, 0=no)')
plt.show()                                       

## Section 2

In [3]:
#start spark session
spark = SparkSession.builder.appName("LogisticRegression").getOrCreate()

#load data from file
spark_df = spark.read.load("C:/Users/usman/Desktop/Master/Big Data Analytics/Assignment 2/Dataset/kindle_reviews.csv", format="csv", sep=",", header="true", inferSchema=True)


In [4]:
#check to check if data loaded correctly
spark_df.head(3)

#remove rows with empty values
spark_df = spark_df.dropna(subset=['reviewText','overall'])

In [5]:
#splits the reviewtext column into seperate words and a new column
# code sourced and adapted from: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.Tokenizer.html
tokenizer = Tokenizer(inputCol='reviewText',outputCol='text')

#removes stop words in and store in new column
# code sourced and adpated from https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.StopWordsRemover.html
remover = StopWordsRemover(inputCol='text',outputCol='filteredText')

#changed the filtered text to numerical
# code sourced and adapted from: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.HashingTF.html
hashing = HashingTF(inputCol='filteredText',outputCol='numberedText',numFeatures=5000)


In [6]:
idf = IDF(inputCol='numberedText',outputCol='features')

indexer = StringIndexer(inputCol='overall',outputCol='label')

spark_df = indexer.fit(spark_df).transform(spark_df)

In [7]:
#split the data into training sets (80%) and validation sets (20%)
[trainingData, testData] = spark_df.randomSplit([0.8, 0.2])

In [8]:
#start the logistic regression model
lr = LogisticRegression(featuresCol='features',labelCol='label')

In [9]:
#combines all steps performed above into one order
pipeline = Pipeline(stages=[tokenizer, remover, hashing, idf, lr])

In [10]:
#train the lr model using the pipeline and training data
model = pipeline.fit(trainingData)

#test the model using the test data
prediction = model.transform(testData)

In [11]:
evaluator = MulticlassClassificationEvaluator(labelCol='label',predictionCol='prediction', metricName='accuracy')

accuracy=evaluator.evaluate(prediction)

print("Classification Error %g"%(1.0-accuracy))

Classification Error 0.35081


In [12]:
#stop the spark session
spark.stop()